# Image Classification: More Pets
 Trains a model to classify an image as a rabbit, mouse, hamster, fish, lizard, or snake.

Below we do the following:

1. Load images of rabbits, mic, hamsters, fish, lizards, and snakes.
2. Train an image classifier model.
3. Convert the model to CoreML and upload it to Skafos.

## Data Preparation and Model Training
The training data for this example are hundreds of images of various animals, pulled from the [Open Images Dataset v4](https://storage.googleapis.com/openimages/web/index.html). 

After unzipping and extracting the images, they are loaded into a Turi Create SFrame and labels are created for each image based on the path. The data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. At the moment, this training environment only supports CPU training.

In [ ]:
# Import libraries
import os
import urllib
import tarfile

import coremltools
import turicreate as tc

In [ ]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/MorePets.tar.gz"
data_path = "MorePets.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
# Load images - you can ignore various jpeg decode errors
data = tc.image_analysis.load_images('MorePets', with_path=True, ignore_failure=True)

# From the path-name, create a label column. This labels each image as the appropriate plant
data['label'] = data['path'].apply(lambda path: os.path.basename(os.path.dirname(path)))

In [ ]:
# Make a train-set split
train_data, test_data = data.random_split(0.8)

In [ ]:
# Let's take a look at some training data
train_data.head()

In [ ]:
# Train an image classification model - consider increasing max_iterations
model = tc.image_classifier.create(
    dataset=train_data,
    target='label',
    model='resnet-50',
    batch_size=32,
    max_iterations=20
)

# Image Classification Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.image_classifier.create.html#turicreate.image_classifier.create

## Model Evaluation

In [ ]:
# Let's see how the model performs on the hold out test data
predictions = model.predict(test_data)
accuracy = tc.evaluation.accuracy(test_data['label'], predictions)
print(f"Image classifer is {accuracy*100} % accuracte on the testing dataset", flush=True)

## Model Export and Skafos Upload
- Convert the model to CoreML format so that it can run on an iOS device. Then deliver the model to your apps with **[Skafos](https://skafos.ai)**.

- If you don't already have an account, Sign Up for one **[here](https://dashboard.skafos.ai)**. 
- Once you've signed up for an account, grab an API token from your account settings.

In [ ]:
# Specify the CoreML model name
model_name = 'ImageClassifier'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name)

In [ ]:
import skafos
from skafos import models
import os

# Set your API Token first for repeated use
os.environ["SKAFOS_API_TOKEN"] = "<YOUR-SKAFOS-API-TOKEN>"
skafos.summary()

In [ ]:
# You can retrieve this info with skafos.summary()
org_name = "<YOUR-SKAFOS-ORG-NAME>"    # Example: "mike-gmail-com-467h2"
app_name = "<YOUR-SKAFOS-APP-NAME>"    # Example: "ImageClassification-App"
model_name = "<YOUR-MODEL-NAME>"       # Example: "ImageClassificationModel"

# Upload model version to Skafos
model_upload_result = models.upload_version(
    files="ImageClassifier.mlmodel",
    org_name=org_name,
    app_name=app_name,
    model_name=model_name
)